# Pipeline Testing

In [6]:
import torch
import pytorch_lightning as pl
import yaml
import importlib
from LightningModules.Embedding.LayerlessEmbedding import LayerlessEmbedding
from LightningModules.Processing.Feature_Construction import Feature_Store

import warnings
warnings.filterwarnings('ignore')

## Data Loading

In [2]:
with open("LightningModules/Processing/prepare_feature_store.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
preprocess_dm = Feature_Store(config)

In [ ]:
preprocess_dm.prepare_data()

## Model Loading

In [7]:
with open("LightningModules/Embedding/train_embedding_RHCPW.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [8]:
model = LayerlessEmbedding(config)

In [9]:
trainer = pl.Trainer(fast_dev_run=True)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [10]:
trainer.fit(model)


  | Name      | Type       | Params
-----------------------------------------
0 | layers    | ModuleList | 1 M   
1 | emb_layer | Linear     | 4 K   
2 | norm      | LayerNorm  | 1 K   
3 | act       | Tanh       | 0     


Epoch 0:  50%|█████     | 1/2 [00:03<00:03,  3.17s/it, loss=0.734, v_num=10, train_loss=0.734]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2/2 [00:11<00:00,  5.93s/it, loss=0.734, v_num=10, train_loss=0.734]
                                                         

Saving latest checkpoint..


Epoch 0: 100%|██████████| 2/2 [00:12<00:00,  6.08s/it, loss=0.734, v_num=10, train_loss=0.734]


1